In [1]:
#dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#import api key
from config import weather_api_key

#incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#output file (csv)
output_data_file = "output_data/cities.csv"

#range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)
lat_range = (-90, -18)
lng_range = (-180, -150)

#formatting for bold text
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [5]:
#printing out the minneapolis json to see the data
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={weather_api_key}&q="
response = requests.get(query_url + "minneapolis").json()
response

{'coord': {'lon': -93.26, 'lat': 44.98},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 256.74,
  'feels_like': 249.95,
  'temp_min': 255.15,
  'temp_max': 258.15,
  'pressure': 1030,
  'humidity': 77},
 'visibility': 16093,
 'wind': {'speed': 4.6, 'deg': 170},
 'clouds': {'all': 40},
 'dt': 1579617479,
 'sys': {'type': 1,
  'id': 4984,
  'country': 'US',
  'sunrise': 1579614231,
  'sunset': 1579647856},
 'timezone': -21600,
 'id': 5037649,
 'name': 'Minneapolis',
 'cod': 200}

In [2]:
#List for holding lat_lngs and cities
lat_lngs = []
cities = []

#create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=-18.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=-150.000, size=1500)
lat_lngs = zip(lats, lngs)

# Add a one second interval between queries to stay within API query limits
time.sleep(1)

#identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

11

In [7]:
#trying to merge Vallie & Melissa cell 3

#base url
url = "http://api.openweathermap.org/data/2.5/weather?"
#build partial query url
query_url = f"{url}appid={weather_api_key}&q="

city_info = []

#in order to print the print log according to how the homework looks (change comment before final draft)
i = 1
set_number = 1

print("Beginning Data Retrieval") 
print("--------------------------------")

for j, city in enumerate(cities):
    
    if (j % 50 == 0 and j >= 50):
        set_number = set_number + 1
        i = 0
    print(f"Processing Record {i} of Set {set_number} | {city}")
    i=i+1
   
    response = requests.get(query_url + city).json()
    
    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    try:
        lat = response['coord']['lat']
        name = response['name']
        cloudiness = response['clouds']['all']
        country = response['sys']['country']
        date = response['dt']
        humidity = response['main']['humidity']
        lngs = response['coord']['lon']
        max_temp = response['main']['temp_max']
        wind_speed = response['wind']['speed']
        city_info.append({"City": name,
                          "Cloudiness": cloudiness,
                          "Country": country,
                          "Date": date,
                          "Humidity": humidity,
                          "Lat": lat,
                          "Lng": lngs,
                          "Max Temp": max_temp,
                          "Wind Speed": wind_speed})
    except:
        print("City " + color.BOLD + "not " + color.END + "found. Skipping...")
        pass

print("------------------------")
print("Data Retrieval Complete")
print("------------------------")

Beginning Data Retrieval
--------------------------------
Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | avarua
Processing Record 4 of Set 1 | alofi
Processing Record 5 of Set 1 | avera
Processing Record 6 of Set 1 | moerai
Processing Record 7 of Set 1 | tevaitoa
Processing Record 8 of Set 1 | haapiti
Processing Record 9 of Set 1 | neiafu
Processing Record 10 of Set 1 | pangai
Processing Record 11 of Set 1 | vaitape
------------------------
Data Retrieval Complete
------------------------


In [9]:
city_df = pd.DataFrame(city_info)
#city_column = city_df["City"]
print(city_df)

        City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0      Vaini          20      TO  1579617637        94 -21.20 -175.20   
1    Mataura         100      NZ  1579617639        94 -46.19  168.86   
2     Avarua         100      CK  1579617640        94 -21.21 -159.78   
3      Alofi          11      NU  1579617641        93 -19.06 -169.92   
4      Avera          40      US  1579617642        63  33.19  -82.53   
5     Moerai          89      PF  1579617643        84 -22.43 -151.33   
6   Tevaitoa         100      PF  1579617644        71 -16.78 -151.50   
7    Haapiti         100      PF  1579617645        78 -17.57 -149.87   
8     Neiafu          20      TO  1579617646        54 -18.65 -173.98   
9     Pangai           0      TO  1579617648        72 -19.80 -174.35   
10   Vaitape         100      PF  1579617649        75 -16.52 -151.75   

    Max Temp  Wind Speed  
0     295.15        4.09  
1     287.59        0.76  
2     297.15        1.50  
3     292.15   

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
city_info = []
counter = 0
for city in cities:
    city_url = url + "&q=" + city
    counter = counter + 1
    time.sleep(1)
    try:
        json_data = requests.get(city_url).json()
        lat = json_data["coord"]["lat"]
        lon = json_data["coord"]["lon"]
        wind = json_data["wind"]["speed"]
        temp_max = json_data["main"]["temp_max"]
        humidity = json_data["main"]["humidity"]
        city_info.append({
            "Lat": lat,
            "Lon": lon,
            "wind": wind,
            "temp_max": temp_max,
            "humidity": humidity
        })
        print(f"Processing Record {counter} of Set {counter} | {city}")
    except:
        print("Not found")
        pass
print("Done")
city_df = pd.DataFrame(city_info)
lat_df = city_df["Lat"]
lon_df = city_df["Lon"]
max_temp_df = city_df["temp_max"]
humidity_df = city_df["humidity"]
wind_df = city_df["wind"]
print(city_df)
#city_df.to_csv(output_data_file, index_label="City_ID")


Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 2 | mataura
Processing Record 3 of Set 3 | avarua
Processing Record 4 of Set 4 | alofi
Processing Record 5 of Set 5 | avera
Processing Record 6 of Set 6 | moerai
Processing Record 7 of Set 7 | tevaitoa
Processing Record 8 of Set 8 | haapiti
Processing Record 9 of Set 9 | neiafu
Processing Record 10 of Set 10 | pangai
Processing Record 11 of Set 11 | vaitape
Done
      Lat     Lon   wind  temp_max  humidity
0  -21.20 -175.20   9.15     71.60        94
1  -46.19  168.86   1.70     57.99        94
2  -21.21 -159.78   5.82     75.20        94
3  -19.06 -169.92   1.12     66.20       100
4   33.19  -82.53   8.05     32.00        63
5  -22.43 -151.33  16.15     79.39        84
6  -16.78 -151.50   8.21     81.34        71
7  -17.57 -149.87   2.24     80.60        74
8  -18.65 -173.98   7.36     68.00        94
9  -19.80 -174.35   7.99     77.47        72
10 -16.52 -151.75   7.99     81.28        75


In [ ]:
#perform a weather check on each city using a series of successive API calls.
#include a print log of each city as it's being processed (with the city number and city name).

#base url
url = "http://api.openweathermap.org/data/2.5/weather?"
#build partial query url
query_url = f"{url}appid={weather_api_key}&q="

#create lists to store city info that I'm interested in
city_names = []
cloudiness = []
country = []
date = []
humidity = []
city_lats = []
city_lngs = []
max_temp = []
wind_speed = []
city_numbers = []

#trying to store data in this list in order to create a dataframe as I loop through (???)
city_info = []

#in order to print the print log according to how the homework looks (change comment before final draft)
i = 1
set_number = 1



print("Beginning Data Retrieval") 
print("--------------------------------")

for j, city in enumerate(cities):
    
    if (j % 50 == 0 and j >= 50):
        set_number = set_number + 1
        i = 0
    print(f"Processing Record {i} of Set {set_number} | {city}")
    i=i+1
   
    response = requests.get(query_url + city).json()
    
    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    try:
        city_names.append(city)
        city_numbers.append(response['id'])
        city_lats.append(response['coord']['lat'])
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        city_lngs.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
        city_info.append({"City": [city_names],
                "Cloudiness": cloudiness,
                "Country": country,
                "Date": date,
                "Humidity": humidity,
                "Lat": city_lats,
                "Lng": city_lngs,
                "Max Temp": max_temp,
                "Wind Speed": wind_speed})
      
    except:
        print("City " + color.BOLD + "not " + color.END + "found. Skipping...")
        pass

print("------------------------")
print("Data Retrieval Complete")
print("------------------------")

#trying to see what I've created, what I'm working with, what's going wrong!!!
print(city_info)
type(city_info)

In [ ]:
#make a dataframe that includes the info that I've just found
#all the columns are correct, but obviously there are lists inside lists?
city_df = pd.DataFrame(city_info)
city_df

In [ ]:
#try creating a dataframe another way
#looks to work, though temps are incorrect
weather_dict = {"City": city_names,
                "Cloudiness": cloudiness,
                "Country": country,
                "Date": date,
                "Humidity": humidity,
                "Lat": city_lats,
                "Lng": city_lngs,
                "Max Temp": max_temp,
                "Wind Speed": wind_speed}
weather_data = pd.DataFrame(weather_dict)
weather_data

In [ ]:
#export city data into a csv file
#make into a dataframe

city_df.to_csv(output_data_file, index_label="City_ID")
#weather_data.to_csv(output_data_file, index_label="City_ID")
#census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

